In [1]:
from parlay_system import *

# Creating a MoneyLine object
A moneyline is represented as either a positive or a negative number

### Positive MoneyLine
Positive moneyline returns represent the underdog if the odds are +165 that means that you bet 100 to win  165 for a total return of 265. This can be calculated as follows:

#### Calculating profit

$ profit = BET * \frac{ODDS}{100} $

$ profit = 100 * \frac{165}{100} $

$ profit = 165 $

#### Calculating payout

$ payout = BET + profit $

$ payout = 100 + 165 $

$ payout = 265 $

The **MoneyLine()** class takes an event name, bet amount, and odds and will calculate this information for us

In [2]:
pos_ml = MoneyLine(event="positive", bet_amount=100, odds=165)
pos_ml.print_stats()

      event  bet_amount  odds  multiplier  payout
0  positive         100   165        2.65   265.0


### Negative MoneyLine
Negative moneyline returns represent the **favorite** if the odds are -150 that means that you bet 150 to win 100 for a total return of 166. This can be calculated as follows:

#### Calculating profit

$ profit = BET * \bigl(\frac{100}{ODDS} \bigr) $

$ profit = 100 * \bigl(\frac{100}{150} \bigr) $

$ profit = 66.67 $

#### Calculating payout

$ payout = BET + profit $

$ payout = 100 + 66.67 $

$ payout = 166.67 $

In [3]:
neg_ml = MoneyLine(event="negative", bet_amount=100, odds=-150)
neg_ml.print_stats()

      event  bet_amount  odds  multiplier      payout
0  negative         100  -150    1.666667  166.666667


# Creating a Parlay object
A parlay object combines the multipliers from two moneyline objects to create an even hgher return (with more risk)

Here is how you can calculate a "True Odds" Parlay

Let's say we want a parlay with the following teams:
pacers              -150
celtics             +170
bucks               -120


First, we have to determine what the multipliers would be for each game, simply divide what the total payout would be (risk + win) by the risk amount

In [4]:
pacers = MoneyLine(event="pacers", bet_amount=100, odds=-150)
celtics = MoneyLine(event="celtics", bet_amount=100, odds=170)
bucks = MoneyLine(event="bucks", bet_amount=100, odds=-120)

In [5]:
pacers.print_stats()

    event  bet_amount  odds  multiplier      payout
0  pacers         100  -150    1.666667  166.666667


In [6]:
celtics.print_stats()

     event  bet_amount  odds  multiplier  payout
0  celtics         100   170         2.7   270.0


In [7]:
bucks.print_stats()

   event  bet_amount  odds  multiplier      payout
0  bucks         100  -120    1.833333  183.333333


### Parlay Formula
So to create a parlay for these three events. You simply multiple the three multipliers by each other:

$ {parlay\_multiplier} = pacers_{multiplier} * celtics_{multiplier} * bucks_{multiplier} $


$ {parlay\_multiplier} = 1.67 * 2.7 * 1.83 $

$ {parlay\_multiplier} = 8.35 $ 

Now we will use the **Parlay()** class to create a parlay from these three moneylines

In [8]:
parlay = Parlay(money_line_arr=[pacers, celtics, bucks], bet_amount=100)

In [9]:
# Note: This result will be slightly off due to rounding error
parlay.print_stats()

                  event  bet_amount   odds  multiplier  payout
0  pacers_celtics_bucks         100  725.0        8.25   825.0


# Verifying our math with Draftkings
These two classes are necessary for the next step which is to build a system that maximizes return while minimizing risk, by leveraging a combination of parlays. The last step us double chacking that the values calculated by these classes match the values that are giving on draftkings. 

In [10]:
bulls = MoneyLine(event="bulls", bet_amount=100, odds=200)
mavericks = MoneyLine(event="mavericks", bet_amount=100, odds=525)
rockets = MoneyLine(event="rockets", bet_amount=100, odds=-560)

In [11]:
bulls.print_stats()

   event  bet_amount  odds  multiplier  payout
0  bulls         100   200         3.0   300.0


![title](img/one_team_parlay.png)

In [12]:
two_team_parlay = Parlay(
    money_line_arr=[bulls, mavericks], 
    bet_amount=100)

two_team_parlay.print_stats()

             event  bet_amount    odds  multiplier  payout
0  bulls_mavericks         100  1775.0       18.75  1875.0


![title](img/two_team_parlay.png)

In [13]:
three_team_parlay = Parlay(
    money_line_arr=[bulls, mavericks, rockets], 
    bet_amount=100)

three_team_parlay.print_stats()

                     event  bet_amount         odds  multiplier       payout
0  bulls_mavericks_rockets         100  2109.821429   22.098214  2209.821429


![title](img/three_team_parlay.png)

### Notes on the discrpancy
As you can see, the Parlay() class came up with a result of 2209.82 while draftkings calculated a result of 2212.50 for the same parlay. This is likely due to a rounding error as they probably have more fractional values for each moneyline. For the purposes of this analysis well assume that they are the same. The minute differences shouldn't make a differece

# Creating a Parlay System
Here is how I plan to use the concept of parlays. 

Consider the following two matchups between nba teams:

Game1: WAS Wizards(+185) vs DET Pistons(-225)
Game2: DAL Mavericks(+525) vs MIL Bucks(-715)

In [14]:
### Create the 4 moneylines
wizards = MoneyLine(event="wizards", bet_amount=100, odds=185)
pistons = MoneyLine(event="pistons", bet_amount=100, odds=-225)

mavericks = MoneyLine(event="mavericks", bet_amount=100, odds=525)
bucks = MoneyLine(event="bucks", bet_amount=100, odds=-715)

### Set up the moneylines as binaries
binaries = [
    [wizards, pistons],
    [mavericks, bucks]
]

### Step 1: Create all combinations
With these binaries, we could make 4 potential parlays:
```
    parlay_0 = wizards + mavericks
    parlay_1 = wizards + bucks
    parlay_2 = pistons + mavericks
    parlay_3 = pistons + bucks
```

So lets turn them into __Parlay()__ objects

In [15]:
parlay_0 = Parlay(
    money_line_arr=[wizards, mavericks], 
    bet_amount=100)

parlay_1 = Parlay(
    money_line_arr=[wizards, bucks], 
    bet_amount=100)

parlay_2 = Parlay(
    money_line_arr=[pistons, mavericks], 
    bet_amount=100)

parlay_3 = Parlay(
    money_line_arr=[pistons, bucks], 
    bet_amount=100)

In [16]:
# Combine all parlays into a table for easy viewing
all_parlay_df = pd.DataFrame()
all_parlay_df = pd.concat([
    all_parlay_df, 
    parlay_0.statistics, 
    parlay_1.statistics, 
    parlay_2.statistics, 
    parlay_3.statistics
])

In [17]:
# Add a column for 'profit' which is payout minus the cost of betting on all parlays
all_parlay_df['profit'] = all_parlay_df.payout - sum(all_parlay_df['bet_amount'])
all_parlay_df

,event,bet_amount,odds,multiplier,payout,profit
0,wizards_mavericks,100,1681.250000,17.812500,1781.250000,1381.250000
0,wizards_bucks,100,224.860140,3.248601,324.860140,-75.139860
0,pistons_mavericks,100,802.777778,9.027778,902.777778,502.777778
0,pistons_bucks,100,64.646465,1.646465,164.646465,-235.353535


As a reminder, __profiit is calculated as payout minus sum of all_bet_amounts__.

### Interpreting The Initial Results
As you can see here, __if we bet \\$100 on each of the potential outcomes, in 2 out of 4 outcomes, we would make money__. Unfortunately, however, the two outcomes that make money are the least likely outcomes. So as it stands this would not be a very intelligent strategy.

### Goal - Find Maximum average profit (given a few constraints)
The ultimate goal is to find a solution of four input bets where the profit of all __four__ strategies is > 0. Even if it is just \\$0.01, if all four options are \\$0.01 then we just increase the bet amount 10x or 100x and in that case we are guarenteed to make money.

Put differently, we want to __find the "bet_amount" coefficients for the four parlays where "profit" > 1 and the average profit is at its peak.__ 

Using Microsoft Excel's solver this can be done like so:

![title](img/excel_demo.png)

What this tells us is that given these constraints:
```
    bet_amount >= 0.10
    profit >= 0
```
and maximizing the "average_profit" you could make the bets above and in __all of the potential outcomes except "pistons_bucks" make >= \\$0__.

Again, its worth noting that the one instance in which you __lose__ money, is the most likely scenario __(both favorites winning)__

### Replicating Excel with Nonlinear Solver in Python
While excel is very useful in this sense, it would be more ideal to be able to run this solver directly in python so that we can expand this analysis to 3, 4, or even 5 team parlays without having to go back and fourth between python and excel. 

The following is the scipy's solver algorithm using the [SLSQP method](https://en.wikipedia.org/wiki/Sequential_quadratic_programming) (vs the GRG Nonlinear from excel). The SLSQP method just works better from the different ones that I tried.


In [18]:
# x represents an array of input values. In this case, the 4 bet_amount 
# coefficients

all_parlays = [
    parlay_0, 
    parlay_1, 
    parlay_2, 
    parlay_3
]


# Optimization function to be passed into optimize.minimize()
def f(bet_arr):
    parlay_profits = []

    for i in range(len(bet_arr)):
        parlay = all_parlays[i]
        profit = (bet_arr[i] * parlay.multiplier + bet_arr[i]) - sum(bet_arr)
        parlay_profits.append(profit)

    # the return value below will be MINIMIZED
    # we want to MAXIMIZE average profits we take the inverse 
    return -statistics.mean(parlay_profits)

In [19]:
# Create bounds for the bet_amount variables
bnds = ()
bounds = (0.10, 30)
for i in range(len(all_parlays)):
    bnds += (bounds,)
bnds

((0.1, 30), (0.1, 30), (0.1, 30), (0.1, 30))

In [20]:
target_profit = 0.1
cons = [{'type': 'ineq', 'fun': lambda x, i=i : x[i] * all_parlays[i].multiplier - sum(x) - target_profit } for i in range(len(all_parlays))]

# COBYLA doesn't support bounds in this format
FinalVal= optimize.minimize(f, np.ones(len(all_parlays)), method='SLSQP', bounds=bnds, constraints=cons)
print(FinalVal.x)

[0.1        0.24447918 0.1        0.35829859]


In [21]:
# Arrays to store data frame values in
events = []
bets = []
multipliers = []
payouts = []
profits = []

for i in range(len(FinalVal.x)):
    bet = FinalVal.x[i]
    parlay = all_parlays[i]
    event = parlay.event
    multiplier = parlay.multiplier
    payout = bet * parlay.multiplier
    profit = bet * parlay.multiplier - sum(FinalVal.x)
    
    events.append(event)
    bets.append(round(bet, 2))
    multipliers.append(multiplier)
    payouts.append(round(payout, 4))
    profits.append(round(profit, 2))
    
df = pd.DataFrame({'event': events,
                   'bet': bets,
                   'mult': multipliers,
                   'payout': payouts,
                   'profit': profits
                    })
# df = df.sort_values(by=['profit'], ascending=False)
df

,event,bet,mult,payout,profit
0,wizards_mavericks,0.10,17.812500,1.7812,0.98
1,wizards_bucks,0.24,3.248601,0.7942,-0.01
2,pistons_mavericks,0.10,9.027778,0.9028,0.10
3,pistons_bucks,0.36,1.646465,0.5899,-0.21


As a reminder, here are the values that the excel solver calculated for comparison:

![title](img/excel_demo.png)

## ParlaySystem() class runs the optimizations
So now that this has been established I will just mention one more class that exists. This is the class __ParlaySystem()__:
```
### Set up the moneylines as binaries
binaries = [
    [wizards, pistons],
    [mavericks, bucks]
]

ParlaySystem(binaries=binaries,
             target_profit=0.50,
             bounds=(0.01, 30),
            )
```

## Next Steps / Summary

So overall the plan is to do the following steps once given a list of binaries:

1. Create all potential subset combinations from the list of binaries
2. turn the selected subsets binaries into a system of parlays
2. Run the SQSLP solver function to find the optimal bet_amount for each parlay
5. Merge the results from each respective binary subset to evaluate the ones most likely to produce profit
6. Compare with actual past results

__In the next notebook I'll combine all of these and show some results with actual ncaa tournament data__